In [1]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
asdasdasasdas_garbage_classification_path = kagglehub.dataset_download('asdasdasasdas/garbage-classification')

print('Data source import complete.')


Data source import complete.


In [ ]:
import os
import torch
import torchvision
from torch.utils.data import random_split
import torchvision.models as models
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, models, transforms
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import numpy as np
import torch
import torch.nn as nn
import torchvision
from torchvision import datasets
from torch.utils.data import DataLoader, random_split
from torchinfo import summary
import torch.optim as optim


from pathlib import Path
import random
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

Upload the dataset

In [ ]:

image_path = Path('/kaggle/input/garbage-classification/Garbage classification/Garbage classification')

print(image_path)

In [ ]:
for root, dirs, files in os.walk("/kaggle/input"):
    print(root)

I checked classes of trashes classified in dataset

In [ ]:
data_dir  = '/kaggle/input/garbage-classification/Garbage classification/Garbage classification'
classes = os.listdir(data_dir)
print(classes)

# Transform the data to structure appropied for analysis.

**Resize** - 256x256 pixels.

**ToTensor()**
Converts a PIL (or NumPy) image to a PyTorch tensor of type torch.


FloatTensor Scales pixel values from [0, 255] → [0.0, 1.0] by dividing by 255./ float value


Converts a shape from (height, width, channels) → (channels, height, width).

In [ ]:
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms

transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

dataset = ImageFolder(data_dir, transform=transform)

# Dataset structure and examples
I checked how many imagines is in dataset, show a sample of foto and printe the structure of dataset - how many of each classess of trashes is in the presented dataset

In [ ]:
%matplotlib inline

def show_sample(img, label):
    print("Label:", dataset.classes[label], "(Class No: "+ str(label) + ")")
    plt.imshow(img.permute(1, 2, 0))

img, label = dataset[1520]
show_sample(img, label)

In [ ]:
print(f"Number of imagines: {len(dataset)}")

In [ ]:
from collections import Counter

labels = [label for _, label in dataset]
class_counts = Counter(labels)

for i, count in class_counts.items():
    class_name = dataset.classes[i]
    print(f"Class '{class_name}': {count} imagines")

I split data set into train, test and validation data

In [ ]:
full_dataset = datasets.ImageFolder(data_dir, transform=transform)

total_size = len(full_dataset)
train_size = int(0.7 * total_size)
val_size = int(0.1 * total_size)
test_size = total_size - train_size - val_size


train_dataset, val_dataset, test_dataset = random_split(full_dataset, [train_size, val_size, test_size])


# DataLoader - a interesting formula for "packing" the imagines in designed bath size structure (in my case 32 imagine per package)
Why we can/should used it?

* Loading data in batches - instead of feeding the model one image at a time, you load a group (e.g. 32 images at once), which speeds up training.

* Random mixing of data (shuffle) - so that the model does not learn the order, which may be random, but sees the data in a random order (important for training).

* Easy iteration over data - DataLoader is iterable, so you can easily write training loops.

* Loading data in the background (multi-threading) - DataLoader can use multiple threads to prepare subsequent batches, so as not to wait for disk.

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)

print(f"Train samples: {len(train_dataset)}")
print(f"Validation samples: {len(val_dataset)}")
print(f"Test samples: {len(test_dataset)}")

# An example of single batch

In [ ]:
from torchvision.utils import make_grid

def show_batch(dl):
    for images, labels in dl:
        fig, ax = plt.subplots(figsize=(12, 6))
        ax.set_xticks([])
        ax.set_yticks([])
        ax.imshow(make_grid(images, nrow = 16).permute(1, 2, 0))
        break

show_batch(train_loader)

# Load pre-trained model

The ResNet50 model was trained on the massive ImageNet dataset and “learned” to detect basic patterns and features in images (e.g. edges, textures)


In [ ]:
model = models.resnet50(pretrained=True)

# Freezing layers

1. Faster training and lower hardware requirements

You don't have to calculate gradients for the entire network - you save memory and time.

2. Transfer learning - using pre-trained features

If your problem is similar (e.g. image classification), you often don't need to change the entire network, just adapt it to your classes.

By freezing the layers, you preserve these "trained" features and only train the last layer, which adapts the output to your classes.

3. Prevents overfitting with small amounts of data

If you have few images, then training the entire network from scratch can lead to overfitting - the model will "remember" the data instead of learning general patterns.

Freezing most of the layers allows you to train fewer parameters and be more stable.

In [ ]:
for param in model.parameters():
    param.requires_grad = False

Because we have a classification problem with garbage classification we need to adjust the last layer for amount of classes we have in our dataset. Garbage classification contains 6 classess of diffrent trashes.

In [ ]:
num_classes = 6
model.fc = nn.Linear(model.fc.in_features, num_classes)

Transfer the model to GPU if available

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

What we have here - we creates a loss function..

* CrossEntropyLoss is a typical function for  used in multi-class classification.
* optimizer = optim.Adam(model.fc.parameters(), lr=0.001)
Creates an optimizer that updates the model parameters after each iteration to minimize the loss.

We use Adam - a modern optimizer, better than classic SGD in most cases.

model.fc.parameters() means that we optimize only the last layer (fc), and the rest of the network (frozen earlier) is not trained.
lr=0.001 is the learning rate - how fast the model learns.

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

In [ ]:
model.train()
for images, labels in train_loader:
    images, labels = images.to(device), labels.to(device)

    optimizer.zero_grad()
    outputs = model(images)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    print(f"Loss: {loss.item():.4f}")

In [ ]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Validation accuracy: {100 * correct / total:.2f}%")

# ~80 % accuracy -not bad but I prefer a better outcome !

# An improved approach

RandomResizedCrop(224) Crops a random fragment and scales it to 224×224 px. Increases crop diversity — the model doesn't get used to a fixed composition.

RandomHorizontalFlip() Randomly flips the image horizontally. Helps when left/right doesn't matter (e.g. junk).

RandomRotation(10) Rotates the image randomly by max 10°. The model learns to be resistant to slight rotations.

ColorJitter(...) Changes brightness, contrast, saturation, hue. The model doesn't learn only colors — shape, texture are more important.

ToTensor() Converts the image from PIL → PyTorch Tensor (0–1). Tensor is a format required by PyTorch.

Normalize(...) Normalizes RGB values (mean/std from ImageNet). ResNet50 was trained on such normalized images — it needs to get data in the same format.


In [ ]:
train_transform_imp = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(0.2, 0.2, 0.2, 0.1),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

In [ ]:
val_test_transform_imp = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])


In [ ]:
full_dataset_imp = datasets.ImageFolder(data_dir, transform=train_transform_imp)
num_classes = len(full_dataset_imp.classes)
print(num_classes)

In [ ]:
train_size_imp = int(0.7 * len(full_dataset_imp))
val_size_imp = int(0.15 * len(full_dataset_imp))
test_size_imp = len(full_dataset_imp) - train_size_imp - val_size_imp
train_dataset_imp, val_dataset_imp, test_dataset_imp = random_split(full_dataset_imp, [train_size_imp, val_size_imp, test_size_imp])

**For validation and test, we don't want to change images randomly. We want to know:**

"Does the model classify data that is as it is in real life?"

From now on, validation (and test) will use different, more relaxed transformations" — e.g., without augmentation.

In [ ]:
val_dataset_imp.dataset.transform = val_test_transform_imp
test_dataset_imp.dataset.transform = val_test_transform_imp

In [ ]:
train_loader_imp = DataLoader(train_dataset_imp, batch_size=32, shuffle=True)
val_loader_imp   = DataLoader(val_dataset_imp, batch_size=32)
test_loader_imp  = DataLoader(test_dataset_imp, batch_size=32)

In [ ]:
model_imp = models.resnet50(pretrained=True)

**Unlock layers layer3, layer4**

In [ ]:
for name, param in model_imp.named_parameters():
    if "layer3" in name or "layer4" in name or "fc" in name:
        param.requires_grad = True
    else:
        param.requires_grad = False

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

In [ ]:
in_features = model_imp.fc.in_features
model_imp.fc = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(in_features, 256),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(256, num_classes)
)

model_imp = model_imp.to(device)


In [ ]:
criterion_imp = nn.CrossEntropyLoss()
optimizer_imp = optim.Adam(filter(lambda p: p.requires_grad, model_imp.parameters()), lr=1e-4)
scheduler_imp = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

In [ ]:
def evaluate(model_imp, loader):
    model_imp.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model_imp(images)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    return correct / total

In [ ]:
num_epochs =15

train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []
test_losses = []
test_accuracies = []


def evaluate_loss_acc(model_imp, loader, criterion_imp, device):
    model_imp.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model_imp(images)
            loss = criterion_imp(outputs, labels)
            running_loss += loss.item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    avg_loss = running_loss / total
    accuracy = correct / total
    return avg_loss, accuracy

for epoch in range(num_epochs):
    model_imp.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    for images, labels in train_loader_imp:
        images = images.to(device)
        labels = labels.to(device)

        optimizer_imp.zero_grad()
        outputs = model_imp(images)
        loss = criterion_imp(outputs, labels)
        loss.backward()
        optimizer_imp.step()

        running_loss += loss.item() * images.size(0)
        _, preds = torch.max(outputs, 1)
        correct_train += (preds == labels).sum().item()
        total_train += labels.size(0)

    train_loss = running_loss / total_train
    train_acc = correct_train / total_train

    test_loss, test_acc = evaluate_loss_acc(model_imp, test_loader_imp, criterion_imp, device)
    val_loss, val_acc = evaluate_loss_acc(model_imp, val_loader_imp, criterion_imp, device)

    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_accuracies.append(train_acc)
    val_accuracies.append(val_acc)
    test_losses.append(test_loss)
    test_accuracies.append(test_acc)


    scheduler_imp.step()

    print(f"Epoch [{epoch+1}/{num_epochs}] "
          f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} | "
          f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f} | "
          f"Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}")

In [ ]:
model_imp.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in val_loader_imp:
        images, labels = images.to(device), labels.to(device)
        outputs = model_imp(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Validation accuracy: {100 * correct / total:.2f}%")

In [ ]:
model_imp.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader_imp:
        images, labels = images.to(device), labels.to(device)
        outputs = model_imp(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy on test data: {100 * correct / total:.2f}%")

In [ ]:
import torchvision.utils as vutils

def show_predictions(model_imp, loader, classes, device, n=16):
    model_imp.eval()
    images, labels = next(iter(loader))
    images, labels = images.to(device), labels.to(device)
    outputs = model_imp(images)
    _, preds = torch.max(outputs, 1)

    images = images.cpu()
    fig = plt.figure(figsize=(15, 6))
    for idx in range(min(n, len(images))):
        ax = fig.add_subplot(2, n//2, idx+1, xticks=[], yticks=[])
        img = images[idx]
        img = img.permute(1, 2, 0)
        img = img * torch.tensor([0.229, 0.224, 0.225]) + torch.tensor([0.485, 0.456, 0.406])
        img = img.clamp(0, 1)
        ax.imshow(img.numpy())
        ax.set_title(f"P: {classes[preds[idx]]}\nT: {classes[labels[idx]]}",
                     color=("green" if preds[idx]==labels[idx] else "red"))
    plt.show()


show_predictions(model_imp, val_loader_imp, full_dataset_imp.classes, device)

In [ ]:
epochs_range = range(1, num_epochs + 1)

plt.figure(figsize=(14, 5))

# Loss plot
plt.subplot(1, 2, 1)
plt.plot(epochs_range, train_losses, label='Train Loss')
plt.plot(epochs_range, test_losses, label='Test Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss over Epochs')
plt.legend()
plt.grid(True)

# Accuracy plot
plt.subplot(1, 2, 2)
plt.plot(epochs_range, train_accuracies, label='Train Accuracy')
plt.plot(epochs_range, test_accuracies, label='Test Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Accuracy over Epochs')
plt.legend()
plt.grid(True)

plt.show()


# Normal CNN

In [ ]:
data_dir

transform_CNN = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

full_dataset_CNN = datasets.ImageFolder(data_dir, transform=transform_CNN)

In [ ]:
train_size_CNN = int(0.7 * len(full_dataset_CNN))
val_size_CNN = int(0.15 * len(full_dataset_CNN))
test_size_CNN = len(full_dataset_CNN) - train_size_CNN - val_size_CNN

train_dataset_CNN, val_dataset_CNN, test_dataset_CNN = random_split(full_dataset_CNN, [train_size_CNN, val_size_CNN, test_size_CNN])

train_loader_CNN = DataLoader(train_dataset_CNN, batch_size=64, shuffle=True)
val_loader_CNN = DataLoader(val_dataset_CNN, batch_size=64)
test_loader_CNN = DataLoader(test_dataset_CNN, batch_size=64)

In [ ]:
criterion_CNN = nn.CrossEntropyLoss()
optimizer_CNN = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
class SimpleCNN(nn.Module):
    def __init__(self, num_classes_CNN):
        super(SimpleCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )

        with torch.no_grad():
            dummy = torch.zeros(1, 3, 64, 64)
            dummy = self.features(dummy)
            print("Shape after features:", dummy.shape)  # np. torch.Size([1, 128, 8, 8])
            n_features = dummy.shape[1] * dummy.shape[2] * dummy.shape[3]

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(n_features, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes_CNN)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

num_classes_CNN = len(full_dataset_CNN.classes)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_CNN = SimpleCNN(num_classes_CNN).to(device)

criterion_CNN = nn.CrossEntropyLoss()
optimizer_CNN = optim.Adam(model_CNN.parameters(), lr=1e-3)


In [ ]:
num_epochs_CNN = 15

train_losses_CNN = []
val_losses_CNN = []
train_accuracies_CNN = []
val_accuracies_CNN = []
test_losses_CNN = []
test_accuracies_CNN = []

for epoch in range(num_epochs_CNN):
    model_CNN.train()
    running_loss_CNN = 0.0
    correct_train = 0
    total_train = 0

    for images, labels in train_loader_CNN:
        images, labels = images.to(device), labels.to(device)

        optimizer_CNN.zero_grad()
        outputs = model_CNN(images)
        loss = criterion_CNN(outputs, labels)
        loss.backward()
        optimizer_CNN.step()

        running_loss_CNN += loss.item() * images.size(0)
        _, predicted = torch.max(outputs, 1)
        correct_train += (predicted == labels).sum().item()
        total_train += labels.size(0)

    model_CNN.eval()
    val_correct_CNN = 0
    val_total_CNN = 0
    with torch.no_grad():
        for images, labels in val_loader_CNN:
            images, labels = images.to(device), labels.to(device)
            outputs = model_CNN(images)
            _, predicted = torch.max(outputs, 1)
            val_correct_CNN += (predicted == labels).sum().item()
            val_total_CNN += labels.size(0)

    train_loss_CNN = running_loss_CNN / total_train
    train_acc_CNN = correct_train / total_train


    test_loss_CNN, test_acc_CNN = evaluate_loss_acc(model_CNN, test_loader_CNN, criterion_CNN, device)
    val_loss_CNN, val_acc_CNN = evaluate_loss_acc(model_CNN, val_loader_CNN, criterion_CNN, device)

    train_losses_CNN.append(train_loss_CNN)
    val_losses_CNN.append(val_loss_CNN)
    train_accuracies_CNN.append(train_acc_CNN)
    val_accuracies_CNN.append(val_acc_CNN)
    test_losses_CNN.append(test_loss_CNN)
    test_accuracies_CNN.append(test_acc_CNN)

    print(f"Epoch {epoch+1}/{num_epochs_CNN}, Loss: {train_loss_CNN:.4f}, Train Acc: {train_acc_CNN:.4f}, Val Acc: {val_acc_CNN:.4f},, Test Acc: {test_acc_CNN:.4f}")


In [ ]:
epochs_range_CNN = range(1, num_epochs + 1)

plt.figure(figsize=(14, 5))

# Loss plot
plt.subplot(1, 2, 1)
plt.plot(epochs_range_CNN, train_losses_CNN, label='Train Loss')
plt.plot(epochs_range_CNN, test_losses_CNN, label='Test Loss')
plt.plot(epochs_range_CNN, val_losses_CNN, label='Test Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss over Epochs')
plt.legend()
plt.grid(True)

# Accuracy plot
plt.subplot(1, 2, 2)
plt.plot(epochs_range_CNN, train_accuracies_CNN, label='Train Accuracy')
plt.plot(epochs_range_CNN, test_accuracies_CNN, label='Test Accuracy')
plt.plot(epochs_range_CNN, val_accuracies_CNN, label='Test Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Accuracy over Epochs')
plt.legend()
plt.grid(True)

plt.show()

In [ ]:
model_CNN.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in val_loader_CNN:
        images, labels = images.to(device), labels.to(device)
        outputs = model_CNN(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Validation accuracy CNN: {100 * correct / total:.2f}%")

In [ ]:
model_CNN.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader_CNN:
        images, labels = images.to(device), labels.to(device)
        outputs = model_CNN(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy on test data CNN: {100 * correct / total:.2f}%")